In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# for Google Colab
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#panel = pd.read_pickle('data sources/returns_chars_panel.pkl') # all the stocks over all the time
#file too large, download it into D disk.
#macro = pd.read_pickle('data sources/macro_timeseries.pkl')   # macro indices


#panel = pd.read_pickle("/content/drive/MyDrive/BDFII_Assignment/Data/returns_chars_panel.pkl") # stock characteristics
macro = pd.read_pickle("/content/drive/MyDrive/Big_Data_Fin2/Data/macro_timeseries.pkl")    # macro indices

In [ ]:
pd.set_option('display.max_rows', None)
panel_descriptives = panel.describe()
print(panel_descriptives.T[['count','min','50%','max']])

                     count           min           50%           max
permno           3739449.0  10000.000000  61778.000000  93436.000000
excess_ret       3739449.0     -1.012517     -0.003450     23.996800
ret              3739449.0     -1.000000      0.000000     24.000000
rfree            3739449.0      0.000008      0.004117      0.013583
mvel1            3739449.0     -0.999781      0.001033      1.000000
beta             3739449.0     -0.999195      0.000000      1.000000
betasq           3739449.0     -0.999755      0.000000      1.000000
chmom            3739449.0     -0.999200      0.000000      1.000000
dolvol           3739449.0     -0.999777      0.000000      1.000000
idiovol          3739449.0     -0.999755      0.000000      1.000000
indmom           3739449.0     -0.999291      0.000000      1.000000
mom1m            3739449.0     -0.999191      0.000000      1.000000
mom6m            3739449.0     -0.999185      0.000000      1.000000
mom12m           3739449.0     -0.

In [8]:
macro_descriptives = macro.describe()
print(macro_descriptives.T[['count','min','50%','max']])

            count       min       50%       max
dp         1118.0 -4.523640 -3.359242 -1.873246
ep         1118.0 -4.836478 -2.797407 -1.774952
b/m        1118.0  0.120510  0.534786  2.028478
crsp_spvw  1118.0 -0.287461  0.012887  0.414312
svar       1118.0  0.000072  0.001246  0.070945
tbl        1118.0  0.000100  0.029000  0.163000
tms        1118.0 -0.036500  0.017500  0.045500
dfy        1118.0  0.003200  0.009000  0.056400
dfr        1118.0 -0.097500  0.000500  0.073700
ntis       1118.0 -0.055953  0.016609  0.177040
infl       1118.0 -0.020548  0.002402  0.058824


In [ ]:
macro.head()

,date,dp,ep,b/m,crsp_spvw,svar,tbl,tms,dfy,dfr,ntis,infl
671,1926-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,1927-01-01,-2.973012,-2.386837,0.441476,0.026047,0.000465,0.0307,0.0047,0.0100,-0.0022,0.050876,0.000000
673,1927-02-01,-2.942374,-2.374773,0.443706,-0.002910,0.000470,0.0323,0.0028,0.0095,-0.0019,0.050824,-0.011299
674,1927-03-01,-2.979535,-2.430353,0.428501,0.045522,0.000287,0.0329,0.0018,0.0092,-0.0019,0.051668,-0.005714
675,1927-04-01,-2.976535,-2.445079,0.469765,0.007324,0.000924,0.0320,0.0011,0.0092,-0.0170,0.046357,-0.005747


In [ ]:
# combine micro and macro data
# In the paper, the variables [crsp_spvw, dfr, infl] are not used
df = pd.merge(panel,macro.drop(columns=['crsp_spvw','dfr','infl']),on='date',how='left',suffixes=['','_macro']) 
df.shape

(3739449, 107)

In [ ]:
# to save the storing space
del panel, macro

In [ ]:
df['permno'].nunique()

29853

In [ ]:
# there are 84 industrial codes (first two digits of the original codes) 
df['permno'] = df['permno'].astype('str').str[:2]

inds = df['permno'].unique()
print(inds)
print(len(inds))

['10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '20' '21' '22' '23'
 '24' '25' '26' '27' '28' '29' '30' '31' '32' '33' '34' '35' '36' '37'
 '38' '39' '40' '41' '42' '43' '44' '45' '46' '47' '48' '49' '50' '51'
 '52' '53' '54' '55' '56' '57' '58' '59' '60' '61' '62' '63' '64' '65'
 '66' '67' '68' '69' '70' '71' '72' '73' '74' '75' '76' '77' '78' '79'
 '80' '81' '82' '83' '84' '85' '86' '87' '88' '89' '90' '91' '92' '93']
84


In [ ]:
# convert the industrial code to dummies and merge to the dataframe. Keep k-1 ind dummies to avoid multi-colinearity
ind_dummies = pd.get_dummies(df['permno'],prefix="ind",drop_first=True)
df = pd.concat([df,ind_dummies], axis=1)
df.shape

(3739449, 190)

In [ ]:
df.to_pickle("data sources/correct dataset.pkl")  